In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

urls=pd.read_csv("./report_urls.csv", encoding="cp949")
urls

url_dic={}
for url in range(len(urls)):
    url_dic[urls.loc[url, "company"]]=urls.loc[url, "url"]

In [7]:
report=pd.read_csv("./report_hyperlinks.csv")
report.dropna(subset = ["name", "page"], axis = 0, how="all", inplace=True)
report.reset_index(inplace = True, drop = True)

#목차 페이지
page=[int(report.loc[idx, "name"].split("=")[1].split("&")[0]) if np.isnan(report.loc[idx, "page"])
      else int(report.loc[idx, "page"]) for idx in range(len(report))]
report["page"]=page

df=report[["text", "company", "page"]].sort_values(['company', 'page'])
df=df.drop_duplicates()
df.reset_index(inplace=True, drop=True)
df

,text,company,page
0,NaN,BGF리테일,1
1,NaN,BGF리테일,3
2,OUR BUSINESS\n,BGF리테일,4
3,CEO Message �\n5 \n,BGF리테일,5
4,CEO Message \n,BGF리테일,5
...,...,...,...
14113,95 Independent Auditor's Report \n,휠라홀딩스,95
14114,103\n,휠라홀딩스,103
14115,105�Greenhouse�Gas�Verification�Statement\n,휠라홀딩스,105
14116,105 Greenhouse Gas Verification Statement\n,휠라홀딩스,105


In [8]:
#보고서 페이지 column
df['page_e'] = 200

for i in tqdm(range(len(df) - 1)) :
    if df['company'][i] == df['company'][i + 1] :
        check = 1
        while(True) :
            if df['page'][i] == df['page'][i + check] : check += 1
            else :
                df['page_e'][i] = df['page'][i + check]
                break

            if i + check == len(df) : break

df

100%|██████████| 14117/14117 [00:02<00:00, 5201.78it/s]


,text,company,page,page_e
0,NaN,BGF리테일,1,3
1,NaN,BGF리테일,3,4
2,OUR BUSINESS\n,BGF리테일,4,5
3,CEO Message �\n5 \n,BGF리테일,5,6
4,CEO Message \n,BGF리테일,5,6
...,...,...,...,...
14113,95 Independent Auditor's Report \n,휠라홀딩스,95,103
14114,103\n,휠라홀딩스,103,105
14115,105�Greenhouse�Gas�Verification�Statement\n,휠라홀딩스,105,106
14116,105 Greenhouse Gas Verification Statement\n,휠라홀딩스,105,106


In [5]:
!pip install fitz
!pip install PyMuPDF

In [13]:
import fitz
import requests

# 페이지 로드 함수 정의
def load_page(company, pages):
    # PDF 파일을 열고 페이지를 읽어 텍스트 추출
    with fitz.open(stream=requests.get(url_dic[company], stream=True).content) as pdf:
        text = []
        for start_page, end_page in set(pages):
            for page in range(start_page, end_page):
                text.append(pdf.load_page(page).get_text().replace("\n", ""))
    return text

# 기업 별로 환경, 사회, 지배 구분하여 페이지 목록을 로드하는 함수
e_content, s_content, g_content = [], [], []

for company in set(df["company"]):
    print(company)
    company_df = df[df["company"] == company]
    
    # 'text' 열이 float일 경우에 대한 처리
    company_df["text"] = company_df["text"].apply(lambda x: str(x).lower() if isinstance(x, str) else "")
    
    # 환경(e), 사회(s), 지배(g) 페이지 목록 생성
    e_page, s_page, g_page = [], [], []
    for idx, x in company_df.iterrows():
        text = x["text"]
        if "environment" in text or "환경" in text or text == 'e':
            e_page.append((x["page"], x["page_e"]))
        elif "social" in text or "사회" in text or (text == 's' and '이사회' not in text):
            s_page.append((x["page"], x["page_e"]))
        elif "gover" in text or "지배" in text or text == 'g':
            g_page.append((x["page"], x["page_e"]))
    print(set(e_page))

    # 보고서 내용을 각각의 리스트에 추가
    e_content.extend(load_page(company, set(e_page)))
    s_content.extend(load_page(company, set(s_page)))
    g_content.extend(load_page(company, set(g_page)))

report_list = [e_content, s_content, g_content]

삼성중공업
{(44, 46), (120, 121)}
삼양패키징
{(19, 20), (20, 24), (24, 27)}
KB금융
{(65, 66), (152, 155), (76, 78), (66, 67), (82, 86), (78, 82), (20, 21)}
두산에너빌리티
{(24, 27), (23, 24)}
미래에셋생명
{(31, 32), (27, 28), (16, 17), (28, 31)}
하이브
{(44, 46), (87, 88)}
현대백화점
{(42, 43), (50, 51), (45, 46)}
두산퓨얼셀
{(44, 45), (24, 25), (28, 29), (23, 24)}
넥센타이어
{(24, 25), (25, 26), (32, 36)}
HDC현대산업개발
{(21, 23), (18, 20), (26, 27), (17, 18), (25, 26)}
한화시스템
{(26, 27), (19, 20), (95, 96), (20, 22), (13, 15)}
하이트진로
{(21, 22), (77, 78), (31, 32), (22, 26), (29, 30)}
넷마블
{(46, 47), (40, 46), (90, 91)}
삼성전기
{(45, 46), (36, 37)}
ESR켄달스퀘어리츠
{(23, 27), (57, 58)}
롯데웰푸드
{(76, 84)}
에스케이바이오팜
{(35, 36), (38, 42), (32, 34)}
현대건설
{(50, 51), (47, 48)}
LG디스플레이
{(17, 18), (111, 112), (55, 56)}
엔씨소프트
{(61, 62), (35, 36), (37, 38), (41, 42), (40, 41), (38, 40), (36, 37)}
BNK금융지주
{(48, 49), (52, 59), (49, 52)}
롯데정보통신
{(23, 24), (24, 33)}
모나리자
{(33, 42), (26, 30), (25, 26), (42, 47)}
고려아연
{(26, 27), (20, 23), (31, 33)}
한화투자증권
{(42, 45

In [14]:
#뉴스 내용
news=pd.read_csv("./krx_news.csv", encoding="cp949")

news_list = []
for i in ["환경", "사회", "지배구조"]:
    text = news[news["news_clss_nm"] == i]
    total_text = sum([news.split("다.") for news in text["contents"]], [])
    news_list.append(total_text)


In [12]:
news_list[1]

['영상\n\n영상\n\n\n\n포토\n\n포토\n\n\n\n정치\n\n대통령실\n국회·정당\n통일·외교\n국방·안보\n북한\n한민족·다문화\n\n\n\n경제\n\n경제일반\n경제정책\n지표\n공정거래\n농림수산\n공기업\n과학기술\n\n\n\n사회\n\n사건·사고\n법원·검찰\n재난\n교육\n복지\n노동\n환경\n교통\n여성·아동\n서울시\n날씨\n인사\n부고\n\n\n\n글로벌\n\n특파원\n글로벌정치\n글로벌경제\n미국·북미\n중남미\n일본\n유럽\n동남아·호주\n베트남\n인도\n중동\n아프리카\n러시아\n동구\n기타\n\n\n\n중국\n\n정치\n경제일반\n정책\n마켓·금융\n산업\n기업\n신기술\n뉴비즈\n문화\n피플\n컬럼\n한중동향\n그래픽\n韓流中心\n\n\n\n산업\n\n재계·경영\n전기·전자\n자동차\n항공\n물류\n철강\n조선\n화학\n중공업\n에너지\n상사\nICT\n중기·벤처\n생활경제\n제약·바이오\n\n\n\n증권·금융\n\n은행\n증권\n보험\n카드\n시황\n주식\n펀드\n채권·외환\n가상통화\nVC\n저축은행·대부업\n핀테크\n\n\n\n부동산\n\n정책\n건설\n분양\n수익형부동산\n\n\n\n전국\n\n지자체\n단체장\n강원\n경기남부\n경기북부\n부산·울산·경남\n대구·경북\n충북\n광주·전남\n대전·충남·세종\n전북\n인천\n제주\n\n\n\n문화·연예\n\n문화일반\n문화정책\n공연\n전시·아트\n출판\n종교\n미디어\n드라마·예능&쇼\n영화\n가요\n패션뷰티\n국내 연예\n해외 연예\n\n\n\n스포츠\n\n해외스포츠\n국내스포츠\n축구\n야구\n골프\n\n\n\n오피니언\n\n오피니언',
 '속보=강원랜드가 폐광기금 부과처분 취소소송에 대한 법원의 화해조정 권고를 수용하지 않자(본지 4월6일자 5면) 태백지역사회가 소송 중단을 촉구하며 강력 반발하고 있',
 '\n\r\n태백지역 93개 사회단체 대표들로 구성된 태백시지역현안대책위원회(이하 현대위)는 6일 강원도와 강원랜드, 산업통상자원부에 보낸 성명서를 통해 “폐기금

In [15]:
#보고서 + 뉴스 내용 명사토큰만
from ekonlpy import Mecab
mecab=Mecab()

def tokenize(content):
    pos=[]
    for x in content:
        p= mecab.pos(x)
        a=[i[0] for i in p if i[1]=="NNG"]
        pos.append(" ".join(a))
    return pos

e_pos=tokenize(report_list[0])
s_pos=tokenize(report_list[1])
g_pos=tokenize(report_list[2])

In [21]:
report_list[0][5]

'환경안전경영체계환경안전경영 비전 및 전략한화시스템은 안전하고 쾌적한 직장과 건강한 지역사회를 위하여 모든 경영 단계에서 환경, 안전, 보건을 지켜 나가기 위해 전사적인관리체계를 강화하고 있습니다.성숙한 S.H.E (Safety, Health, Environmental) 문화 정착을 통해회사의 지속적인 성장과 발전에 기여하고 지속 가능한 안전보건 경영시스템 구축, 안전하고 건강한 사업장의 유지 및 관리, 지속적인 환경개선 및 에너지 효율 향상을 목표로 사회적, 환경적 책임과 의무를 다하기 위하기 위한 노력을 이어가겠습니다.환경안전경영 거버넌스한화시스템은 환경 안전 비전과 전략에 따라 임직원의 안전과  건강을 위한 사회적 책임을 다하기 위하여 환경안전보건위원회를 개최하고 있으며, 환경안전보건 전담 실무 조직을 기존 팀에서 안전경영실로 승격하여 환경안전경영 거버넌스 체계를 고도화 하였습니다. 기업의 사회적 책임 강화 요구와 중대재해처벌법 시행에 발맞추어 사업장 내 안전 문화의 정착을 위하여 기업의 최고 의사 결정자인 CEO가 안전 보건 업무의 총 책임자로서 책임과 의무를 다하고 있습니다.환경안전관리 규정 및 제도한화시스템은 환경안전 정책 또는 방침 등을 수립하여 공개함으로써 환경 문제 해결에 대한 경영진의 의지를 전사적으로 전달 할 수 있도록 하고 있습니다. 또한 외부 이해관계자(협력업체 포함)에게도 한화시스템 환경안전정책을 공유 함으로써 환경문제에 대한 당사의 노력과 신뢰를  주고 있습니다. 환경안전경영우리의 기술은 사회 구성원 모두가 안전하고 편안한 내일을 꿈꿀 수 있도록 따뜻함이 가득한, 지속가능한 세상을 만들어 갑니다.구 분전사 환경안전경영 운영체계사내 규정공통• ECO-YHES 제도• 절대안전수칙• 환경안전경영매뉴얼 외 12개• 목표, 경영프로그램 운영환 경• 대기환경보전법 외 24개• 화학물질관리 제도• 재활용 의무 대상 제품 제도• 환경영향 등록 업무 규칙 외 4개환경안전 보건• 산업안전보건법 외 14개• 위험성평가 제도• 환경안전 교육• 환경안전사고

In [16]:
e_news_pos=tokenize(news_list[0])
s_news_pos=tokenize(news_list[1])
g_news_pos=tokenize(news_list[2])

In [17]:
with open("./stopword.txt", "r") as f:
  stopwords=f.read().splitlines()

In [28]:
#TextRank

from collections import Counter, defaultdict

def scan_vocabulary(token_content, min_count=2):
    counter = Counter(content for contents in token_content for content in contents.split(" "))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx


def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i.split(" ") if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)


from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))


def word_graph(token_content, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(token_content, min_count)
    tokens = [contents for contents in token_content]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence )#verbose
    return g, idx_to_vocab


import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

def textrank_keyword(token_content, min_count, window, min_cooccurrence, df, max_iter, topk, stopwords):
    g, idx_to_vocab = word_graph(token_content, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()

    if stopwords is None:
        stopwords = {}
    keywords = {idx_to_vocab[idx]: R[idx] for idx in reversed(idxs) if not (idx_to_vocab[idx] in stopwords) and len(idx_to_vocab[idx]) >= 2}

    if topk > 0:
        keywords = {word:r for word, r in sorted(keywords.items(), key=lambda x:-x[1])[:topk]}

    return keywords

In [30]:
from collections import Counter, defaultdict
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

def scan_vocabulary(token_content, min_count=2):
    # 토큰의 빈도 계산
    counter = Counter(content for contents in token_content for content in contents.split(" "))
    # 최소 빈도 이상인 토큰만 선택
    counter = {w:c for w,c in counter.items() if c >= min_count}
    # 토큰을 빈도순으로 정렬하여 인덱스 부여
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i.split(" ") if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    # 최소 공기 횟수 이상인 연결 관계만 선택
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    # 희소 행렬 생성
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

def word_graph(token_content, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(token_content, min_count)
    tokens = [contents for contents in token_content]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence )
    return g, idx_to_vocab

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # 정규화 및 초기화
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # 반복 수행
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

def textrank_keyword(token_content, min_count=10, window=2, min_cooccurrence=2, df=0.85, max_iter=10, topk=100, stopwords=stopwords):
    g, idx_to_vocab = word_graph(token_content, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()

    if stopwords is None:
        stopwords = {}
    # 중요 단어 및 불용어 필터링
    keywords = {idx_to_vocab[idx]: R[idx] for idx in reversed(idxs) if not (idx_to_vocab[idx] in stopwords) and len(idx_to_vocab[idx]) >= 2}

    if topk > 0:
        # 상위 k개의 키워드 선택
        keywords = {word:r for word, r in sorted(keywords.items(), key=lambda x:-x[1])[:topk]}

    return keywords

In [20]:
ekeywords=textrank_keyword(e_pos, 10, 2, 2, df=0.85, max_iter=10, topk=100, stopwords=stopwords)
skeywords=textrank_keyword(s_pos, 10, 2, 2, df=0.85, max_iter=10, topk=100, stopwords=stopwords)
gkeywords=textrank_keyword(g_pos, 10, 2, 2, df=0.85, max_iter=10, topk=100, stopwords=stopwords)

In [22]:
ekeywords

{'친환경': 19.644991918200606,
 '물질': 16.14023422218102,
 '환경경영': 13.747216147445272,
 '폐기물': 13.175390395157764,
 '에너지': 12.006012495371163,
 '배출': 9.430978390971992,
 '화학': 8.102859914869887,
 '인증': 7.612007787546369,
 '저감': 7.332800524681614,
 '개발': 7.243178379642196,
 '배출량': 7.072406169583274,
 '기후변화': 6.839937141801152,
 '사용량': 6.593915756391694,
 '감축': 6.041543761208854,
 '적용': 5.958316827058693,
 '온실가스': 5.819882998516754,
 '자원': 5.507039162867063,
 '구매': 5.2576180616593655,
 '용수': 5.256531748991411,
 '처리': 5.026286570709859,
 '탄소중립': 4.849321269031635,
 '시설': 4.828242609158242,
 '탄소': 4.821774159687177,
 '최소': 4.725972210710469,
 '절감': 4.720309342925312,
 '유해': 4.4132803824528155,
 '대비': 4.334172281312488,
 '소재': 4.24613788378625,
 '모니터링': 4.170651233692272,
 '설치': 4.1281540272497175,
 '이행': 4.124311637206403,
 '설비': 4.112764728600581,
 '효율': 4.027812833766883,
 '도입': 3.95504916974082,
 '플라스틱': 3.8262611608330808,
 '분석': 3.7856152338072477,
 '법규': 3.7776325505524317,
 '폐수': 3.6930

In [23]:
#명사토큰만 (text_rank)
esg_news_df=pd.concat([pd.DataFrame(ekeywords.items()), pd.DataFrame(skeywords.items()), pd.DataFrame(gkeywords.items())], axis=1)
esg_news_df.columns=["E", "E_score", "S", "S_score", "G", "G_score"]
esg_news_df

,E,E_score,S,S_score,G,G_score
0,친환경,19.644992,인권,11.134858,이사회,28.922252
1,물질,16.140234,이사회,10.831786,사외이사,23.648206
2,환경경영,13.747216,프로그램,9.340271,주주,8.754652
3,폐기물,13.175390,보건,8.712606,선임,8.387428
4,에너지,12.006012,사외이사,7.484534,후보,7.489520
...,...,...,...,...,...,...
95,자체,2.063231,존중,2.225250,선정,1.509672
96,측정,2.060871,학생,2.216828,유지,1.482047
97,비율,2.032691,임원,2.209556,프로세스,1.475821
98,주기,2.026608,연계,2.206510,내부통제,1.451057


In [24]:
#명사토큰(textrank)
esg_report_df=pd.concat([pd.DataFrame(ekeywords.items()), pd.DataFrame(skeywords.items()), pd.DataFrame(gkeywords.items())], axis=1)
esg_report_df.columns=["E", "E_score", "S", "S_score", "G", "G_score"]
esg_report_df

,E,E_score,S,S_score,G,G_score
0,친환경,19.644992,인권,11.134858,이사회,28.922252
1,물질,16.140234,이사회,10.831786,사외이사,23.648206
2,환경경영,13.747216,프로그램,9.340271,주주,8.754652
3,폐기물,13.175390,보건,8.712606,선임,8.387428
4,에너지,12.006012,사외이사,7.484534,후보,7.489520
...,...,...,...,...,...,...
95,자체,2.063231,존중,2.225250,선정,1.509672
96,측정,2.060871,학생,2.216828,유지,1.482047
97,비율,2.032691,임원,2.209556,프로세스,1.475821
98,주기,2.026608,연계,2.206510,내부통제,1.451057


In [25]:
from collections import Counter
keys=list(ekeywords.keys()) + list(skeywords.keys()) + list(gkeywords.keys())
Counter(keys)

Counter({'친환경': 2,
         '물질': 1,
         '환경경영': 1,
         '폐기물': 1,
         '에너지': 2,
         '배출': 1,
         '화학': 1,
         '인증': 1,
         '저감': 1,
         '개발': 2,
         '배출량': 1,
         '기후변화': 2,
         '사용량': 1,
         '감축': 1,
         '적용': 1,
         '온실가스': 1,
         '자원': 1,
         '구매': 2,
         '용수': 1,
         '처리': 2,
         '탄소중립': 1,
         '시설': 2,
         '탄소': 1,
         '최소': 1,
         '절감': 1,
         '유해': 1,
         '대비': 1,
         '소재': 1,
         '모니터링': 2,
         '설치': 1,
         '이행': 3,
         '설비': 1,
         '효율': 1,
         '도입': 1,
         '플라스틱': 1,
         '분석': 1,
         '법규': 1,
         '폐수': 1,
         '보건': 3,
         '데이터': 2,
         '순환': 1,
         '생물': 1,
         '전환': 1,
         '인권': 2,
         '달성': 1,
         '원료': 1,
         '오염': 1,
         '방지': 1,
         '단계': 1,
         '검증': 2,
         '프로세스': 2,
         'ISO': 1,
         '전사': 2,
         '위험': 3,
       

In [20]:
!pip install soynlp

                                              0.0/416.8 kB ? eta -:--:--
                                              10.2/416.8 kB ? eta -:--:--
     -------                               81.9/416.8 kB 919.0 kB/s eta 0:00:01
     -------------------------------------- 416.8/416.8 kB 3.3 MB/s eta 0:00:00


In [26]:
import math
import numpy as np
from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer
from sklearn.metrics import pairwise_distances

def vectorize(sents, vocab_to_idx, tokenize):
    # 문장을 벡터화하여 CSR 행렬로 반환하는 함수
    rows, cols, data = [], [], []
    for i, sent in enumerate(sents):
        terms = set(tokenize(sent))
        for term in terms:
            j = vocab_to_idx.get(term, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(1)
    n_docs = len(sents)
    n_terms = len(vocab_to_idx)
    return csr_matrix((data, (rows, cols)), shape=(n_docs, n_terms))

def make_vocab_score(keywords, scaling=None):
    # 단어별 중요도를 스케일링하여 반환하는 함수
    if scaling is None:
        scaling = lambda x: math.sqrt(x)
    return {word: scaling(rank) for word, rank in keywords.items()}

def select(x, keyvec, texts, initial_penalty, topk=10):
    # 주어진 행렬과 키워드 벡터를 기반으로 주요 문장을 선택하는 함수
    dist = pairwise_distances(x, keyvec, metric='cosine').reshape(-1)
    dist = dist + initial_penalty
    diversity = 0.7
    idxs = []
    for _ in range(topk):
        idx = dist.argmin()
        idxs.append(idx)
        dist[idx] += 2  # 코사인 거리의 최대값은 2
        idx_all_distance = pairwise_distances(x, x[idx].reshape(1, -1), metric='cosine').reshape(-1)
        penalty = np.zeros(idx_all_distance.shape[0])
        penalty[np.where(idx_all_distance < diversity)[0]] = 2
        dist += penalty
    return [texts[idx] for idx in idxs]

# 단어별 중요도를 계산하여 편차를 완화한 후 tokenizer 생성
keywords = make_vocab_score(ekeywords)  
tokenizer = MaxScoreTokenizer(scores=keywords)

# 단어를 인덱스로 매핑하고 벡터화하여 x에 저장
idx_to_vocab = [vocab for vocab in sorted(keywords, key=lambda x: -keywords[x])]
vocab_to_idx = {vocab: idx for idx, vocab in enumerate(idx_to_vocab)}
x = vectorize(e_news_pos, vocab_to_idx, tokenizer.tokenize)  

# 키워드 벡터 생성 후 select 함수를 통해 주요 문장 선택
keyvec = np.asarray([keywords[vocab] for vocab in idx_to_vocab]).reshape(1, -1)
select(x, keyvec, e_news_pos, 1, topk=10)

['통신업 온실가스배출량 태양광 친환경 설비 투자 확대 공동 펀드 조성 탄소 배출량 저감 활동 전력',
 '대기오염 배출 저감 시설 관심 비용 투자 오염 물질 배출 감축 석탄 탄소중립 계획 탄소중립 추진 요구',
 '환경 국장 공장 공공 폐수처리 수 분석 항목 일반 종과 특정 수질 유해 물질 종 종 법 기준치 준수 배출 공개 화학 물질 중앙 지방 정부 후 일정 기간 일반 공개 시스템 구축 시민 불안 해소 효과 수질 대책 수립',
 '녹색기업 환경기술 산업 법 오염 물질 감소 자원 에너지 절감 녹색경영 체제 구축 환경 개선 기여 기업 기관 지정 환경 관련 신고 사항 면제 혜택 제도',
 '지방 통합 환경 허가 조건 이행 여부 대기 수질 배출 시설 방지 시설 적정 설치 운영 여부 폐기물 관리 환경 법령 준수 여부 집중 점검 결과 법령 위반 사항 확인',
 '수소 환원 제철 기술개발 전기로 신설 에너지 효율 개선 탄소 포집 저장 활용 기술 고도화 탄소중립 달성 총력',
 '업계 관계자 온실가스 감축 중요 네트워크 장비 증설 인터넷데이터센터 구축 통신서비스 품질 개선 우선순위 감축목표 기존 설비 에너지 효율 차질 설명',
 '플라스틱 사용량 조사 애플리케이션 개발 데이터 분석 담당 그린피스 데이터 액티비스트 플 조사 참가자 대상 인터넷 설문조사 진행 결과 응답자 플라스틱 문제 해결 방안 기업 과대 포장 포장재 사용 시스템 전환 정부 일회용 플라스틱 사용 규제',
 '홍수 자원 순환 사회 필요 플라스틱 생산 자체 중요 이러 본질 문제 친환경 소재 홍보 말',
 '보고서 석유화학기업 플라스틱 생산 증가 충격 기여도 일 플라스틱 온실가스배출량 영국 전체 배출량 배출량 대부분 플라스틱 생산 단계 발생']

In [27]:
# 스탑워즈 불러오기
with open("./stopword.txt", "r") as f:
    stopwords=f.read().splitlines()

with open("./stopword_2.txt", "r") as f:
    stopwords_=f.read().splitlines()

s_word_e, s_word_s, s_word_g = [], [], []

# 스탑워즈 분리 코드
[s_word_e.append(x.split()[0]) if x.split()[1] == 'E'
  else s_word_s.append(x.split()[0]) if x.split()[1] == 'S'
  else s_word_g.append(x.split()[0]) for x in stopwords_]

len(s_word_e), len(s_word_s), len(s_word_g)

(2, 5, 9)

In [31]:
# 리포트 시드워드 단어 점수 추출
ekeywords,skeywords,gkeywords=[textrank_keyword(x, stopwords=stopwords + s_word)
                               for x, s_word in ((e_pos, s_word_e),(s_pos, s_word_s),(g_pos, s_word_g))]
gkeywords

{'주주': 8.75465173500411,
 '선임': 8.387427773218958,
 '후보': 7.489520379237952,
 '보수': 6.954909509223306,
 '추천': 6.521182682475924,
 '주주총회': 6.108424353546733,
 '리스크관리': 6.048136867356938,
 '투명': 5.900225411851173,
 '독립': 5.866425974681215,
 '감사위원회': 5.821581401548821,
 '위원': 5.219871004415227,
 '지배구조': 5.181742786615853,
 '규정': 5.0393801320690095,
 '재무': 4.9513201273035685,
 '감사': 4.918298136491191,
 '승인': 4.250826432055228,
 '대표이사': 4.117931479966282,
 '경영진': 4.066871577455052,
 '제고': 3.6806336654765497,
 '회계': 3.668154614745886,
 '정기': 3.574558509798033,
 '안건': 3.262617617277194,
 '준법': 3.160577487781891,
 '주식': 3.1407852874980318,
 '공시': 3.0637925091098537,
 '지급': 3.0431746909284922,
 '결의': 3.0062832619592275,
 '임원': 3.0052649290746087,
 '분야': 2.7513520693489806,
 '역할': 2.7165303901491034,
 '당사': 2.693924287095054,
 '의사결정': 2.6680847821538363,
 '심의': 2.665845517933153,
 '거래': 2.6337349045258374,
 '지속가능경영': 2.5862827272602416,
 '의결권': 2.556911078666603,
 '법률': 2.525627452636695,
 '출석':

In [32]:
# 뉴스 시드워드 단어 점수 추출
ekeywords_news,skeywords_news,gkeywords_news=[textrank_keyword(x, stopwords=stopwords + s_word)
                               for x, s_word in ((e_news_pos, s_word_e),(s_news_pos, s_word_s),(g_news_pos, s_word_g))]
gkeywords_news

{'의혹': 27.087438557544935,
 '구속': 23.589759841440806,
 '횡령': 20.417905126432373,
 '부지사': 16.41879183308512,
 '영장': 14.841633012355793,
 '기소': 13.280902896196238,
 '검사': 13.104339073542297,
 '변호사': 12.922440320140327,
 '주식': 11.55548508955652,
 '지검': 11.443703399911797,
 '부장': 11.069988329997843,
 '해외': 10.951570881732978,
 '거래': 10.504415274396711,
 '범죄': 10.48876944347084,
 '재판': 10.088115280015925,
 '대북': 9.973390046385756,
 '송금': 9.159780943933246,
 '자금': 9.085655021934613,
 '뇌물': 8.64753119314523,
 '배임': 8.64355127094942,
 '특검': 8.415742671100539,
 '법인': 8.281534314843329,
 '대납': 7.59601275969565,
 '증거': 7.5711981560593635,
 '진술': 7.24280453115274,
 '마약': 7.225189829827972,
 '북한': 7.178998279719442,
 '지분': 7.151891341186966,
 '중앙': 7.135086035022165,
 '청구': 6.930744975777817,
 '평화': 6.837569392074188,
 '은행': 6.650872547368666,
 '달러': 6.647774811620713,
 '도피': 6.39286626799092,
 '중국': 6.32816760682343,
 '대출': 6.221829051389154,
 '임원': 6.075925434037879,
 '체포': 6.038982419209814,
 '상

In [33]:
# 점수 합치기
res_e, res_s, res_g = {}, {}, {}

res_e.update(ekeywords_news)
res_s.update(skeywords_news)
res_g.update(gkeywords_news)

res_e.update(ekeywords)
res_s.update(skeywords)
res_g.update(gkeywords)

In [34]:
# 데이터 프레임 생성
res = pd.DataFrame()

for d, tag in ((res_e, 'e'), (res_s, 's'), (res_g, 'g')) :
    res = pd.concat([res, pd.DataFrame(d.items(), columns=[f'keyword_{tag}', f'score_{tag}'])], axis = 1)

res

,keyword_e,score_e,keyword_s,score_s,keyword_g,score_g
0,배출,9.430978,노조,34.085561,의혹,27.087439
1,온실가스,5.819883,화재,22.594644,구속,23.589760
2,에너지,12.006012,파업,20.217275,횡령,20.417905
3,플라스틱,3.826261,노동자,17.183269,부지사,16.418792
4,환경부,3.618789,작업,14.608710,영장,14.841633
...,...,...,...,...,...,...
190,NaN,NaN,임원,2.209556,NaN,NaN
191,NaN,NaN,연계,2.206510,NaN,NaN
192,NaN,NaN,이슈,2.198485,NaN,NaN
193,NaN,NaN,프로세스,2.189191,NaN,NaN


In [ ]:
res.to_csv('seed_word_list.csv', index = False, encoding = 'utf-8')